In [1]:
import os
import django
from django.forms.models import model_to_dict

os.chdir('/Users/lavenderca/genomics_network/')
django.setup()

In [2]:
from network import models

#### Clean house

In [3]:
models.GenomeAssembly.objects.all().delete()
models.GeneAnnotation.objects.all().delete()
models.GenomicRegions.objects.all().delete()
models.Gene.objects.all().delete()
models.Transcript.objects.all().delete()

(0, {'network.Transcript': 0})

#### Add assemblies

In [4]:
for assembly in ['hg19', 'mm9']:
    models.GenomeAssembly.objects.create(
        name = assembly,
        default_annotation_id = None
    )

#### Add annotations

In [3]:
curr_dir = os.getcwd()

assembly_to_annotation = {
    'hg19': curr_dir + '/data/annotations/hg19_RefSeq_Genes_2016-10-25.gtf',
    'mm9': curr_dir + '/data/annotations/mm9_RefSeq_Genes_2016-10-25.gtf',
}

assembly_to_table = {
    'hg19': curr_dir + '/data/annotation_tables/hg19_RefSeq_Genes_2016-10-25',
    'mm9': curr_dir + '/data/annotation_tables/mm9_RefSeq_Genes_2016-10-25',
}

In [6]:
for assembly, annotation in assembly_to_annotation.items():
    assembly_obj = models.GenomeAssembly.objects.filter(name__startswith=assembly)[0]
    annotation_obj = models.GeneAnnotation.objects.create(
        name = assembly + '_RefSeq',
        gtf_file = annotation,
        assembly = assembly_obj,
    )
    assembly_obj.default_annotation = annotation_obj

#### Read in GTF transcripts

In [4]:
ACCEPTED_CHROMOSOMES = []

# Works for mouse and human
# No 'random', 'haplo', 'Un', 'Y'
for i in range(1, 22):
    ACCEPTED_CHROMOSOMES.append('chr' + str(i))
ACCEPTED_CHROMOSOMES.append('chrX')

In [5]:
def getTranscriptsFromGTF(gtf_fn):
    transcripts = dict()
    chromosome_set = set()
    
    with open(gtf_fn) as f:
        for line in f:
            line_split = line.strip().split('\t')
            
            chromosome = line_split[0]
            entry_type = line_split[2]
            (start, end) = int(line_split[3]), int(line_split[4])
            strand = line_split[6]
            
            detail = line_split[8]
            
            if chromosome in ACCEPTED_CHROMOSOMES:
                transcript_id = detail.split('transcript_id')[1].split(';')[0].split('"')[1]

                if transcript_id not in transcripts:
                    transcripts[(transcript_id, chromosome)] = {
                        'chromosome': chromosome,
                        'strand': strand,
                        'exons': [],
                    }

                transcripts[(transcript_id, chromosome)]['exons'].append((start, end))
    
    for transcript in transcripts.values():
        transcript['exons'].sort(key=lambda x: x[0])
        transcript['start'] = transcript['exons'][0][0]
        transcript['end'] = transcript['exons'][-1][1]
    
    return transcripts

In [6]:
assembly_to_transcripts = dict()
for assembly, annotation_file in assembly_to_annotation.items():
    assembly_to_transcripts[assembly] = getTranscriptsFromGTF(annotation_file)

#### Create TSS/promoter BED files

In [7]:
from collections import defaultdict

def createTSSBEDFromTranscripts(transcripts, output_file):
    
    tss_dict = defaultdict(set)
    for tr_id, transcript in transcripts.items():
        chromosome = transcript['chromosome']
        strand = transcript['strand']
        if strand == '+':
            start = transcript['start']
        if strand == '-':
            start = transcript['end']
        name = tr_id[0]
        
        tss_dict[(chromosome, strand, start)].add(name)
    
    entry_num = 0
    with open(output_file, 'w') as OUTPUT:
        for tr_id, names in sorted(
                tss_dict.items(),
                key=lambda x: (
                    ACCEPTED_CHROMOSOMES.index(x[0][0]), 
                    x[0][2]
                )
            ):
            
            tr_names = ','.join(sorted(list(names), key=lambda x: int(x.split('_')[1])))
            chromosome = tr_id[0]
            entry_name = '{}|{}|{}'.format(str(entry_num), tr_names, chromosome)
            
            start = tr_id[2]
            strand = tr_id[1]
            
            OUTPUT.write('{}\t{}\t{}\t{}\t{}\n'.format(
                chromosome,
                str(start - 1),
                str(start),
                entry_name,
                '0',
                strand,
            ))
            
            entry_num += 1

In [11]:
createTSSBEDFromTranscripts(assembly_to_transcripts['mm9'], 'temp')

In [8]:
for assembly, transcripts in assembly_to_transcripts.items():
    bed_file = 'data/genomic_regions/' + assembly + '_RefSeq_promoters.bed'
    createTSSBEDFromTranscripts(transcripts, bed_file)

#### Add to database and associate regions to annotations

In [13]:
for annotation, region_file in [
    ('hg19_RefSeq', curr_dir + '/data/genomic_regions/hg19_RefSeq_promoters.bed',),
    ('mm9_RefSeq', curr_dir + '/data/genomic_regions/mm9_RefSeq_promoters.bed',),
]:

    assembly_obj = \
        models.GenomeAssembly.objects.filter(name__startswith=annotation.split('_')[0])[0]
    annotation_obj = models.GeneAnnotation.objects.filter(name__startswith=annotation)[0]
    
    region_obj = models.GenomicRegions.objects.create(
        name = annotation + '_promoters',
        assembly = assembly_obj,
        bed_file = region_file,
    )
    
    annotation_obj.promoters = region_obj

In [14]:
for annotation, region_file in [
    ('hg19_RefSeq', curr_dir + '/data/genomic_regions/hg19_vista_enhancers.bed',),
    ('mm9_RefSeq', curr_dir + '/data/genomic_regions/mm9_vista_enhancers.bed',),
]:

    assembly_obj = \
        models.GenomeAssembly.objects.filter(name__startswith=annotation.split('_')[0])[0]
    annotation_obj = models.GeneAnnotation.objects.filter(name__startswith=annotation)[0]
    
    region_obj = models.GenomicRegions.objects.create(
        name = annotation + '_enhancers',
        assembly = assembly_obj,
        bed_file = region_file,
    )
    
    annotation_obj.promoters = region_obj

#### Add gene and transcripts members

In [15]:
import csv

def getGeneTranscriptAssociations(annotation_table):
    gene_set = set()
    transcript_to_gene = dict()
    
    with open(annotation_table) as f:
        reader = csv.DictReader(f, delimiter='\t')
        for line in reader:
            gene_name = line['name2']
            transcript_name = line['name']
        
            gene_set.add(gene_name)
            transcript_to_gene[transcript_name] = gene_name 
    
    return gene_set, transcript_to_gene

In [16]:
gene_set, transcript_to_gene = \
    getGeneTranscriptAssociations('data/annotation_tables/hg19_RefSeq_Genes_2016-10-25')

In [17]:
for assembly, annotation_table in assembly_to_table.items():
    gene_set, transcript_to_gene = getGeneTranscriptAssociations(annotation_table)
    transcripts = assembly_to_transcripts[assembly]
    
    annotation = assembly + '_RefSeq'
    annotation_obj = models.GeneAnnotation.objects.filter(name__startswith=annotation)[0]
    
    for gene in gene_set:
        models.Gene.objects.create(
            name = gene,
            annotation = annotation_obj,
        )

    for tr_id, transcript in transcripts.items():
        tr_name = tr_id[0].split('_dup')[0]
        associated_gene_name = transcript_to_gene[tr_name]
        associated_gene = \
            models.Gene.objects.filter(name__startswith=associated_gene_name)[0]
        
        tr_obj = models.Transcript.objects.create(
            name = tr_name,
            gene = associated_gene,
            start = transcript['start'],
            end = transcript['end'],
            chromosome = transcript['chromosome'],
            strand = transcript['strand'],
            exons = transcript['exons'],
        )